## 2. Получение предсказаний на тестовой выборке и создание файла для kaggle.

Считаем данные из яндекс-диска

In [1]:
import os
import pandas as pd
path = r'/Users/romanova/Downloads/kaggle_data' 
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("/")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("/")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("/")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("/")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("/")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("/")[-1])
                pass

track_0.csv
track_1.csv
track_10.csv
track_100.csv
track_101.csv
track_102.csv
track_103.csv
track_104.csv
track_105.csv
track_106.csv
track_107.csv
track_108.csv
track_109.csv
track_11.csv
track_110.csv
track_111.csv
track_112.csv
track_113.csv
track_114.csv
track_115.csv
track_116.csv
track_117.csv
track_118.csv
track_119.csv
track_12.csv
track_120.csv
track_121.csv
track_122.csv
track_123.csv
track_124.csv
track_125.csv
track_126.csv
track_127.csv
track_128.csv
track_129.csv
track_13.csv
track_130.csv
track_131.csv
track_132.csv
track_133.csv
track_134.csv
track_135.csv
track_136.csv
track_137.csv
track_138.csv
track_139.csv
track_14.csv
track_140.csv
track_141.csv
track_142.csv
track_143.csv
track_144.csv
track_145.csv
track_15.csv
track_16.csv
track_17.csv
track_18.csv
track_19.csv
track_2.csv
track_20.csv
track_21.csv
track_22.csv
track_23.csv
track_24.csv
track_25.csv
track_26.csv
track_27.csv
track_28.csv
track_29.csv
track_3.csv
track_30.csv
track_31.csv
track_32.csv
track_33.

Переведём типы движений в числа

In [2]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [3]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [ ]:
def rotation_matrix_from_vectors(vec1, vec2):
    a, b = (vec1 / numpy.linalg.norm(vec1)).reshape(3), (vec2 / numpy.linalg.norm(vec2)).reshape(3)
    v = numpy.cross(a, b)
    if any(v): #if not all zeros then 
        c = numpy.dot(a, b)
        s = numpy.linalg.norm(v)
        kmat = numpy.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return numpy.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return numpy.eye(3) #cross of all zeros only occurs on identical directions
    
def rotate(X, Y, Z, Mat):
    new_x = []
    new_y = []
    new_z = []
    
    for elem in zip(X,Y,Z):
        res = Mat.dot(elem)
        new_x.append(res[0])
        new_y.append(res[1])
        new_z.append(res[2])
        
    return new_x, new_y, new_z

def integrate(lst):
    res = []
    s = 0
    for elem in lst:
        s += elem
        res.append(s)
    return res

In [577]:
import pandas as pd
from matplotlib import pylab as plt
%pylab inline

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1):
    
# обрезаем начало и конец трека
    datFram_len = len(df) 
    toCut = round(datFram_len * 0.20)
    if a == -1:
        st = toCut
    else:
        st = a
            
    if b == -1:
        end = -toCut
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1

#    print(name)
#     fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,6))

# приводим данные с Iphone к нужному формату
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return [-1, [-1, -1]]
        
########################################################
# amplitude
########################################################

# вычисляем модуль (норму) ускорения и рисуем его
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
#     ax2.plot(Acc)
#     plt.show()

# можно различить многие действия по амплитуде ускорения
    amplitude = Acc.max() - Acc.min()
#    print(' АМПЛИТУДА:', Acc.max()-Acc.min())
#    print()
    
########################################################
# преобразование Фурье
########################################################

    Fs = 250 #частота сбора данных
    Min = 0
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5

    y = Acc.values
    if len(y) == 0:
        hesitation = -1
    else:
        n = len(y) # length of the signal
        k = np.arange(n)
        T = n/Fs
        frq = k/T # two sides frequency range
        frq = frq[:len(frq)//2] # one side frequency range

        Y = np.fft.fft(y)/n # dft and normalization
        Y = Y[1:n//2]

        yabs = abs(Y)
        xmax = frq[numpy.argmax(yabs)]
        ymax = yabs.max()

        if xmax == 0:
            xmax, ymax = -1, -1
    
#    рисование
#    ax3.plot(frq,yabs) # plotting the spectrum
#    ax3.set_xlim([0,30])
#    ax3.set_ylim([Min,0.1])

########################################################
#    print('Амплитуда:', amplitude, ' Высота:', high, ' Колебания:', hesitation)
    return [amplitude, xmax, ymax]

Information = []
for i in range (len(Data)):
    Information.append(np.zeros(3))

for df, target in zip(Data,Target):
    track_num = int(target.split('_')[1].split('.')[0]) # важный момент, что треки не по нумерации идут!
    Information[track_num] = process_data(df, name=target)

Populating the interactive namespace from numpy and matplotlib


In [578]:
Information

[[2.617663370775875, 4.400570884871551, 0.0945973765543517],
 [1.9935598883036225, 1.898800481902467, 0.08378278697055057],
 [1.854852429103988, 0.42462845010615713, 0.033088463990157586],
 [7.376276708180853, 6.99876492383697, 0.41838871194921273],
 [2.6161380237808616, 0.9116207657614432, 0.09932622647876851],
 [2.9399536075474773, 3.984470780547609, 0.04946176114682434],
 [1.5117563691489093, 1.1240164855751218, 0.04528391403867049],
 [9.43369600102802, 2.690724912947135, 0.45451088753128466],
 [0.9393878301836878, 5.226057365094799, 0.03969777627816896],
 [0.032301908494170695, 1.6566137116642594, 0.00029698579665454153],
 [8.833789142567072, 6.692377495462795, 0.5241492939517084],
 [1.7496778171718206, 1.1803588290840417, 0.03813382172852338],
 [4.364114709035295, 5.91118800461361, 0.3028150392452855],
 [9.793314652916319, 8.179858934169278, 0.24258821024955637],
 [2.4185341114208763, 5.325443786982249, 0.017797856105651804],
 [0.9585710148889119, 0.6480474956100009, 0.05560023842

Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [655]:
def get_prediction(info):
    ampl, xmax, ymax = info[0], info[1], info[2]
    if ampl < 1 and ymax < 0.01:
        return 0 #стояние
    if ampl > 5:
        return 2 #бег
    if 2 < xmax < 4:
        if ymax < 0.1:
            return 4 #лестница
        return 1 #ходьба
    if xmax > 5 and 0.008 < ymax < 0.5:
        return 3 #велосипед
    return -1

In [656]:
len(Data), len(Information), len(Target)

(146, 146, 146)

In [657]:
Result = []

count = 0
for i in range (len(Information)):
    if not ((math.isnan(Information[i][0]) or Information[i][0]==-1) and 
            (math.isnan(Information[i][1]) or Information[i][1]==-1)):
        count += 1
        Result.append(Information[i])
    else:
        Result.append([2, 2, 2])
len(Result), count

(146, 146)

Создадим таблицу, содержащую номер трека и предсказание.

In [661]:
tracks_levels = pd.read_csv('/Users/romanova/Downloads/tracks_levels_3.csv')
somebody = pd.read_csv('/Users/romanova/Downloads/sixth_submit.csv')

In [662]:
for i in range(len(tracks_levels)):
    if tracks_levels['level'][i] == 'advanced':
        prediction[i] = somebody['action'][i]
    else:
        prediction[i] = get_prediction(Result[i])
        if prediction[i]==-1:
            prediction[i] = somebody['action'][i] 

Res = pd.DataFrame({'track_num' : np.arange(len(Data)), 'action' : prediction})
Res.head(10)

,track_num,action
0,0,1
1,1,4
2,2,1
3,3,2
4,4,1
5,5,4
6,6,3
7,7,2
8,8,3
9,9,0


In [663]:
Res['action'].value_counts()

1    40
0    29
2    23
3    22
4    21
6     5
5     4
7     2
Name: action, dtype: int64

In [664]:
Res.to_csv("Romanova_Anastasia_3.csv", index=False)